# All the information is on the task(master fan wiki)

In this notebook we try to use Python's BeautifulSoup module to scrape the wording of all the tasks from series 14 of the popular TV show Taskmaster from the fan wiki:

https://taskmaster.fandom.com/wiki/Series_5

with a view to analysing them.

## The main tasks

In [ ]:
# Ususal uploads

from bs4 import BeautifulSoup
import requests
import numpy as np

from wordcloud import WordCloud
import plotly.express as px
from pprint import pprint
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import pandas as pd

In [ ]:
url = 'https://taskmaster.fandom.com/wiki/Series_5'

In [ ]:
page = requests.get(url)

In [ ]:
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
all_starts = list(soup.find_all("tr", class_='tmtablerow'))
len(all_starts)

In [ ]:
#Putting them all in a single list

all_tasks = []
for i in range(len(all_starts)):
    if len(list(all_starts[i])) > 2:
        all_tasks.append(str(list(all_starts[i])[3]))
    else:
        all_tasks.append(str(list(all_starts[i])[1]))
    
            
pprint(all_tasks)

In [ ]:
# Getting rid of the <td>s from the beinnings 

all_tasks = [task[4:] for task in all_tasks]
all_tasks

In [ ]:
#Remove the stuff from the beginnings

for i in range(len(all_tasks)):
    if 'Prize:' in all_tasks[i]:
        all_tasks[i] = all_tasks[i][14:]
    if 'Team Live' in all_tasks[i]:
        all_tasks[i] = all_tasks[i][18:]
    if 'Team' in all_tasks[i]:
        all_tasks[i] = all_tasks[i][13:]
    if 'Live' in all_tasks[i]:
        all_tasks[i] = all_tasks[i][13:]
        
all_tasks

In [ ]:
# Removing the stuff from the end

all_tasks = [task[:-7] for task in all_tasks]
all_tasks

In [ ]:
all_tasks[13] = all_tasks[13][-12:]
all_tasks[13]

In [ ]:
all_tasks[21] = "Spot the difference. Have a look at the scene in front of you and explain exactly what's changed since you put the blindfold on."
all_tasks[21]

In [ ]:
all_tasks[34] = "Using this flame, light the candle in the caravan."
all_tasks[34]

## NLP

In [ ]:
tasks_split = [task.split() for task in all_tasks]
pprint(tasks_split)

In [ ]:
task_words = []
for task in tasks_split:
    task_words += task

task_words

In [ ]:
tasks_lower = [task.lower() for task in task_words]
tasks_lower

In [ ]:
tasks_no_punc = []

for word in tasks_lower:
    for punctuation in string.punctuation:
        word = word.replace(punctuation, '')
    tasks_no_punc.append(word)
        
tasks_no_punc

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
tasks_no_stop = [word for word in tasks_no_punc if word not in stop_words]

In [ ]:
len(tasks_lower)

In [ ]:
len(tasks_no_stop)

In [ ]:
# Lemmatizing

from nltk.stem import WordNetLemmatizer

# Lemmatizing the verbs
task_verb_lem = [
    WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
    for word in tasks_no_stop
]

task_verb_lem

In [ ]:
tasks_final = [
    WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
    for word in task_verb_lem
]

tasks_final

## Word count

In [ ]:
df = pd.DataFrame(tasks_final)
df

In [ ]:
df[0].value_counts().nlargest(15)

## Word cloud

In [ ]:
from os import path
from PIL import Image
from wordcloud import STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
text = ''
for words in tasks_final:
    text += words + ' '

text

In [ ]:
wordcloud = WordCloud(background_color="white").generate(text)

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()